In [2]:
import pandas as pd
import re
import yaml
import sqlparse
import os
import pandas as pd
import numpy as np
import requests
from IPython.display import display, Markdown

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [3]:
def add_repo_root_path():
    import os
    import sys
    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if repo_root not in sys.path:
        sys.path.append(repo_root)
        
add_repo_root_path()
from src import generate_knowledge
from src import create_rag_db
from src import llm_chain_tools
from src.enhanced_retriever import EnhancedRetriever

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', 10) 

### INIT

In [5]:
generate_knowledge.add_repo_root_path()
import openai_setup

OPENAI_API_KEY = openai_setup.conf['key']
OPENAI_PROJECT = openai_setup.conf['project']
OPENAI_ORGANIZATION = openai_setup.conf['organization']
DEFAULT_LLM_MODEL = "gpt-4o-mini"
CHROMADB_DIRECTORY = '../chromadb'
COLLECTION_NAME = "my_chromadb" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ['OPENAI_MODEL_NAME'] = DEFAULT_LLM_MODEL


In [6]:
langchain_openai_embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")
langchain_openai_llm = ChatOpenAI(model=DEFAULT_LLM_MODEL, temperature=0.1, openai_api_key=OPENAI_API_KEY, openai_organization = OPENAI_ORGANIZATION)

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': '../config/agents.yml',
    'tasks': '../config/tasks.yml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

print(agents_config)
print(tasks_config)

In [8]:
from crewai import Agent, Task, Crew

### TESTS

#### Agents

In [8]:

# Creating Agents
interpretation_agent = Agent(
  config=agents_config['interpretation_agent'],
)

evaluation_agent = Agent(
  config=agents_config['evaluation_agent'],
)

lineage_agent = Agent(
  config=agents_config['lineage_agent'],
)

plan_agent = Agent(
  config=agents_config['plan_agent'],
)

In [9]:
# Creating Tasks
interpretation_task = Task(
  config=tasks_config['interpretation_task'],
  agent=interpretation_agent
)

evaluation_task = Task(
  config=tasks_config['evaluation_task'],
  agent=evaluation_agent
)

lineage_task = Task(
  config=tasks_config['lineage_task'],
  agent=lineage_agent
)

plan_task = Task(
  config=tasks_config['plan_task'],
  agent=plan_agent
)

In [10]:
crew = Crew(
  agents=[
    interpretation_agent,
    evaluation_agent
  ],
  tasks=[
    interpretation_task,
    evaluation_task
  ],
  verbose=True
)

In [ ]:
user_input = 'Give me all the information about the models related with customers'

inputs = {
  'request': user_input
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

#### Flows

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
from crewai import Flow
from crewai.flow.flow import listen, start

class dbtChatFlow(Flow):
    @start()
    def interpret_prompt(self):
        user_prompt = self.state["user_input"]
        print(user_prompt)
        interpretation_result = crew.kickoff(inputs = {'request': user_prompt} )
        self.state["interpretation_result"] = interpretation_result
        return interpretation_result

    @listen(lambda state: "interpretation_result" in state)
    def evaluate_interpretation(self):
        interpretation_result = self.state.get("interpretation_result")
        evaluation_result = crew.agents[1].kickoff({"request": interpretation_result})
        self.state["evaluation_result"] = evaluation_result
        return evaluation_result

flow = dbtChatFlow()
#flow.plot()

In [ ]:
user_input = 'Give me all the information about the models related with customers'
flow.kickoff(inputs={"user_input": user_input})

### CREATE AGENT CHAIN

#### Configure Prerequisites

In [ ]:
from langchain_openai import ChatOpenAI

loaded_vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=CHROMADB_DIRECTORY,
    embedding_function=langchain_openai_embeddings
)

In [10]:
_, repo_name = generate_knowledge.extract_owner_and_repo('https://github.com/dbt-labs/jaffle-shop')
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')
dbt_repo_knowledge_df = create_rag_db.merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df)

In [11]:
retriever = EnhancedRetriever(vectorstore = loaded_vectorstore, embedding_function= langchain_openai_embeddings)

query = "give me all the models related with the dbt model orders"
final_context, top_documents = retriever.retrieve(query)

#### Create agents, tasks and flow

In [12]:
# Creating Agents
interpretation_agent = Agent(
  config=agents_config['interpretation_agent'],
)

evaluation_agent = Agent(
  config=agents_config['evaluation_agent'],
)

lineage_agent = Agent(
  config=agents_config['lineage_agent'],
)

plan_agent = Agent(
  config=agents_config['plan_agent'],
)

In [13]:
# Creating Tasks
interpretation_task = Task(
  config=tasks_config['interpretation_task'],
  agent=interpretation_agent
)

evaluation_task = Task(
  config=tasks_config['evaluation_task'],
  agent=evaluation_agent
)

lineage_task = Task(
  config=tasks_config['lineage_task'],
  agent=lineage_agent
)

plan_task = Task(
  config=tasks_config['plan_task'],
  agent=plan_agent
)

In [ ]:
interpretation_crew = Crew(agents = [interpretation_agent], tasks = [interpretation_task], verbose = True)
evaluation_crew = Crew(agents = [evaluation_agent], tasks = [evaluation_task], verbose = True)
lineage_crew = Crew(agents = [lineage_agent], tasks = [lineage_task], verbose = True)
plan_crew = Crew(agents = [plan_agent], tasks = [plan_task], verbose = True)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from crewai import Flow
from crewai.flow.flow import listen, start

class dbtChatFlow(Flow):
    @start()
    def interpret_prompt(self):
        request = self.state["request"]
        interpretation = interpretation_crew.kickoff(inputs = {'request': request})
        self.state["interpretation"] = interpretation
        return interpretation

    @listen(interpret_prompt)
    def evaluate_interpretation(self):
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        evaluation = evaluation_crew.kickoff(inputs = {'request': request, "interpretation": interpretation})
        self.state["evaluation"] = evaluation
        return evaluation
    
    @listen(evaluate_interpretation)
    def retrieve_general_context_for_lineage_calculation(self):
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        vectorstore = self.state["vectorstore"]
        embedding_function = self.state["embedding_function"]
        retriever = EnhancedRetriever(vectorstore = vectorstore, embedding_function= embedding_function)
        retriever_input = """
            USER REQUEST: {request}
            REQUEST FINALITY: {interpretation}
        """
        retrieved_context, retrieved_documents = retriever.retrieve(retriever_input)
        retrieved_context = "\n".join([doc.page_content for doc in retrieved_documents if hasattr(doc, 'page_content')])
        self.state["retrieved_context"] = retrieved_context
        return retrieved_context

    @listen(retrieve_general_context_for_lineage_calculation)
    def get_lineage(self):
        request = self.state["request"]
        evaluation = self.state.get("evaluation")
        
        retrieved_context = self.state.get("retrieved_context")
        lineage_analysis = lineage_crew.kickoff(inputs = {'request': request, 'evaluation': str(evaluation), 'retrieved_context':retrieved_context})
        json_output = lineage_analysis.raw.replace("```json", "").replace("```", "").strip()
        self.state["lineage_analysis"] = eval(json_output)
        return eval(json_output)
    
    @listen(get_lineage)
    def get_lineage_documents(self):
        lineage_analysis = self.state.get("lineage_analysis")
        vectorstore = self.state["vectorstore"]
        dbt_repo_knowledge_df = self.state["dbt_repo_knowledge_df"]

        model_name = lineage_analysis.get("model")
        scope = lineage_analysis.get("scope", "").upper()

        lineage_df = create_rag_db.plot_dbt_lineage(dbt_repo_knowledge_df)
        affected_models = llm_chain_tools.get_affected_models(lineage_df, model_name)

        if scope == "UP":
            filtered_models = affected_models["upstream"]
        elif scope == "DOWN":
            filtered_models = affected_models["downstream"]
        elif scope == "ALL":
            filtered_models = affected_models["upstream"] + affected_models["downstream"]
        filtered_models = list(set(f"{model}.sql" for model in filtered_models + [model_name]))
        
        documents = llm_chain_tools.extract_documents_from_vectorstore(vectorstore)
        lineage_documents = llm_chain_tools.select_documents(documents, filtered_models)
        self.state["lineage_documents"] = lineage_documents
        return lineage_documents

    @listen(get_lineage_documents)
    def retrieve_lineage_context(self):
        lineage_documents = self.state.get("lineage_documents")
        embedding_function = self.state["embedding_function"]
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        evaluation = self.state.get("evaluation")

        retriever_documents = lineage_documents["retriever_documents"]
        csv_sources_documents = lineage_documents["csv_sources_documents"]
        yml_project_documents = lineage_documents["yml_project_documents"]

        # Create a new vectorstore with the filtered documents
        new_vectorstore = Chroma.from_documents(retriever_documents, embedding_function)
        
        # Adjusted retriever
        new_retriever = EnhancedRetriever(vectorstore = new_vectorstore, embedding_function = embedding_function)
        retriever_input = """
            USER REQUEST: {request}
            REQUEST FINALITY: {interpretation}
            DBT EXPERT DEEP EVALUATION: {evaluation}
        """
        retrieved_context, retrieved_documents = new_retriever.retrieve(retriever_input)
        combined_documents =  yml_project_documents + retrieved_documents

        retrieved_context = "\n".join([doc.page_content for doc in combined_documents if hasattr(doc, 'page_content')])
        retrieved_csv_sources_context = "\n".join([doc.page_content for doc in csv_sources_documents if hasattr(doc, 'page_content')])

        self.state["planning_retrieved_context"] = retrieved_context
        self.state["planning_retrieved_csv_sources_context"] = retrieved_csv_sources_context
        return retrieved_context
    
    @listen(retrieve_lineage_context)
    def plan_changes(self):
        request = self.state["request"]
        evaluation = self.state.get("evaluation")
        lineage_analysis = self.state.get("lineage_analysis")
        planning_retrieved_context = self.state.get("planning_retrieved_context")
        planning_retrieved_csv_sources_context = self.state.get("planning_retrieved_csv_sources_context")

        plan = plan_crew.kickoff(inputs = {'request': request, "evaluation": str(evaluation), "lineage_analysis": str(lineage_analysis), "retrieved_context": planning_retrieved_context,  "retrieved_csv_sources_context":planning_retrieved_csv_sources_context})
        self.state["plan"] = plan
        return plan

flow = dbtChatFlow()
flow.plot()

In [ ]:
user_input = 'Give me all the information about the models related with customers'
result = flow.kickoff(inputs={"request": user_input, "dbt_repo_knowledge_df": dbt_repo_knowledge_df, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})

In [ ]:
Markdown(result.raw)

## ADVANCE CHAIN

In [ ]:
from langchain_openai import ChatOpenAI

loaded_vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=CHROMADB_DIRECTORY,
    embedding_function=langchain_openai_embeddings
)

retriever = EnhancedRetriever(vectorstore = loaded_vectorstore, embedding_function= langchain_openai_embeddings)

_, repo_name = generate_knowledge.extract_owner_and_repo('https://github.com/dbt-labs/jaffle-shop')
dbt_models_df = pd.read_csv('../data/dbt_models_' + repo_name + '.csv')
dbt_project_df = pd.read_csv('../data/dbt_project_' + repo_name + '.csv')
dbt_repo_knowledge_df = create_rag_db.merge_dbt_models_and_project_dfs(dbt_models_df, dbt_project_df)

#### Agents, tasks and crews

In [ ]:
# Creating Agents
check_model_agent = Agent(
  config=agents_config['check_model_agent'],
)

search_model_agent = Agent(
  config=agents_config['search_model_agent'],
)

extract_info_agent = Agent(
  config=agents_config['extract_info_agent'],
)

process_info_agent = Agent(
  config=agents_config['process_info_agent'],
)

solution_design_agent = Agent(
  config=agents_config['solution_design_agent'],
)


In [53]:
# Creating Tasks
check_model_task = Task(
  config=tasks_config['check_model_task'],
  agent=check_model_agent
)

search_model_task = Task(
  config=tasks_config['search_model_task'],
  agent=search_model_agent
)

extract_info_task = Task(
  config=tasks_config['extract_info_task'],
  agent=extract_info_agent
)

process_info_task = Task(
  config=tasks_config['process_info_task'],
  agent=process_info_agent
)

solution_design_task = Task(
  config=tasks_config['solution_design_task'],
  agent=solution_design_agent
)

KeyError: 'check_model_task'

In [52]:
check_model_crew = Crew(agents = [check_model_agent], tasks = [check_model_task], verbose = True)
search_model_crew = Crew(agents = [search_model_agent], tasks = [search_model_task], verbose = True)
extract_info_crew = Crew(agents = [extract_info_agent], tasks = [extract_info_task], verbose = True)
process_info_crew = Crew(agents = [process_info_agent], tasks = [process_info_task], verbose = True)
solution_design_crew = Crew(agents = [solution_design_agent], tasks = [solution_design_task], verbose = True)

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


#### Flow

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from crewai import Flow
from crewai.flow.flow import listen, start

class dbtChatFlow(Flow):
    @start()
    def interpret_prompt(self):
        request = self.state["request"]
        interpretation = interpretation_crew.kickoff(inputs = {'request': request})
        self.state["interpretation"] = interpretation
        return interpretation

    @listen(interpret_prompt)
    def evaluate_interpretation(self):
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        evaluation = evaluation_crew.kickoff(inputs = {'request': request, "interpretation": interpretation})
        self.state["evaluation"] = evaluation
        return evaluation
    
    @listen(evaluate_interpretation)
    def retrieve_general_context_for_lineage_calculation(self):
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        vectorstore = self.state["vectorstore"]
        embedding_function = self.state["embedding_function"]
        retriever = EnhancedRetriever(vectorstore = vectorstore, embedding_function= embedding_function)
        retriever_input = """
            USER REQUEST: {request}
            REQUEST FINALITY: {interpretation}
        """
        retrieved_context, retrieved_documents = retriever.retrieve(retriever_input)
        retrieved_context = "\n".join([doc.page_content for doc in retrieved_documents if hasattr(doc, 'page_content')])
        self.state["retrieved_context"] = retrieved_context
        return retrieved_context

    @listen(retrieve_general_context_for_lineage_calculation)
    def get_lineage(self):
        request = self.state["request"]
        evaluation = self.state.get("evaluation")
        
        retrieved_context = self.state.get("retrieved_context")
        lineage_analysis = lineage_crew.kickoff(inputs = {'request': request, 'evaluation': str(evaluation), 'retrieved_context':retrieved_context})
        json_output = lineage_analysis.raw.replace("```json", "").replace("```", "").strip()
        self.state["lineage_analysis"] = eval(json_output)
        return eval(json_output)
    
    @listen(get_lineage)
    def get_lineage_documents(self):
        lineage_analysis = self.state.get("lineage_analysis")
        vectorstore = self.state["vectorstore"]
        dbt_repo_knowledge_df = self.state["dbt_repo_knowledge_df"]

        model_name = lineage_analysis.get("model")
        scope = lineage_analysis.get("scope", "").upper()

        lineage_df = create_rag_db.plot_dbt_lineage(dbt_repo_knowledge_df)
        affected_models = llm_chain_tools.get_affected_models(lineage_df, model_name)

        if scope == "UP":
            filtered_models = affected_models["upstream"]
        elif scope == "DOWN":
            filtered_models = affected_models["downstream"]
        elif scope == "ALL":
            filtered_models = affected_models["upstream"] + affected_models["downstream"]
        filtered_models = list(set(f"{model}.sql" for model in filtered_models + [model_name]))
        
        documents = llm_chain_tools.extract_documents_from_vectorstore(vectorstore)
        lineage_documents = llm_chain_tools.select_documents(documents, filtered_models)
        self.state["lineage_documents"] = lineage_documents
        return lineage_documents

    @listen(get_lineage_documents)
    def retrieve_lineage_context(self):
        lineage_documents = self.state.get("lineage_documents")
        embedding_function = self.state["embedding_function"]
        request = self.state["request"]
        interpretation = self.state.get("interpretation")
        evaluation = self.state.get("evaluation")

        retriever_documents = lineage_documents["retriever_documents"]
        csv_sources_documents = lineage_documents["csv_sources_documents"]
        yml_project_documents = lineage_documents["yml_project_documents"]

        # Create a new vectorstore with the filtered documents
        new_vectorstore = Chroma.from_documents(retriever_documents, embedding_function)
        
        # Adjusted retriever
        new_retriever = EnhancedRetriever(vectorstore = new_vectorstore, embedding_function = embedding_function)
        retriever_input = """
            USER REQUEST: {request}
            REQUEST FINALITY: {interpretation}
            DBT EXPERT DEEP EVALUATION: {evaluation}
        """
        retrieved_context, retrieved_documents = new_retriever.retrieve(retriever_input)
        combined_documents =  yml_project_documents + retrieved_documents

        retrieved_context = "\n".join([doc.page_content for doc in combined_documents if hasattr(doc, 'page_content')])
        retrieved_csv_sources_context = "\n".join([doc.page_content for doc in csv_sources_documents if hasattr(doc, 'page_content')])

        self.state["planning_retrieved_context"] = retrieved_context
        self.state["planning_retrieved_csv_sources_context"] = retrieved_csv_sources_context
        return retrieved_context
    
    @listen(retrieve_lineage_context)
    def plan_changes(self):
        request = self.state["request"]
        evaluation = self.state.get("evaluation")
        lineage_analysis = self.state.get("lineage_analysis")
        planning_retrieved_context = self.state.get("planning_retrieved_context")
        planning_retrieved_csv_sources_context = self.state.get("planning_retrieved_csv_sources_context")

        plan = plan_crew.kickoff(inputs = {'request': request, "evaluation": str(evaluation), "lineage_analysis": str(lineage_analysis), "retrieved_context": planning_retrieved_context,  "retrieved_csv_sources_context":planning_retrieved_csv_sources_context})
        self.state["plan"] = plan
        return plan

flow = dbtChatFlow()
flow.plot()

In [ ]:
user_input = 'Give me all the information about the models related with customers'
result = flow.kickoff(inputs={"request": user_input, "dbt_repo_knowledge_df": dbt_repo_knowledge_df, "vectorstore": loaded_vectorstore, "embedding_function":langchain_openai_embeddings})
Markdown(result.raw)

#### Example

In [ ]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails